In [58]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [59]:
wd = webdriver.Chrome("./chromedriver.exe") 
wd.implicitly_wait(10)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13968\3653437403.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome("./chromedriver.exe")


In [60]:
wd.get("https://coinmarketcap.com/")


In [61]:
content = wd.page_source.encode("UTF-8").strip()

In [62]:
soup = BeautifulSoup(content, "html.parser")

In [63]:
names = []
prices = []
daily_changes = []
weekly_changes = []

#Get all currencies + fill the names list
coins = soup.select("tbody tr")
for i in range(10):
 coin = coins[i].select("p")
 names.append([coin[1].get_text(), coin[2].get_text()])

 

In [64]:
print(coins[0].prettify())

<tr>
 <td>
  <span class="sc-80eeb0-1 iuAjRY" data-btnname="watchlistStarIcon" data-page="watchlistStarIconPage">
   <span class="icon-Star">
   </span>
  </span>
 </td>
 <td style="text-align:left">
  <p class="sc-1eb5slv-0 etpvrL" color="text2" font-size="0">
   1
  </p>
 </td>
 <td style="text-align:left">
  <div class="sc-16r8icm-0 escjiH" display="flex">
   <a class="cmc-link" href="/currencies/bitcoin/">
    <div class="sc-16r8icm-0 sc-1teo54s-0 dBKWCw">
     <img alt="BTC logo" class="coin-logo" loading="lazy" src="https://s2.coinmarketcap.com/static/img/coins/64x64/1.png"/>
     <div class="sc-16r8icm-0 sc-1teo54s-1 dNOTPP">
      <p class="sc-1eb5slv-0 iworPT" color="text" font-size="1" font-weight="semibold">
       Bitcoin
      </p>
      <div class="sc-1teo54s-2 fZIJcI" data-nosnippet="true">
       <div class="sc-1teo54s-3 etWhyV">
        1
       </div>
       <p class="sc-1eb5slv-0 gGIpIK coin-item-symbol" color="text3" font-size="1">
        BTC
       </p>
      </di

In [65]:
#Get all prices + prices changes
for i in range(10):
 coin = coins[i].select("span")
 prices.append(coin[2].get_text())
 daily_changes.append([coin[5].get_text(), coin[5].find_next("span")["class"][0]])
 weekly_changes.append([coin[7].get_text(), coin[7].find_next("span")["class"][0]])

tbody = """"""

for i in range(10):
 tbody += f"""
 <tr>
  <td>
   {i+1}
  </td>
  <td>
   {names[i][0] + " | " + names[i][1]}
  </td>
  <td>
   {prices[i]}
  </td>
  <td>
   {("U" if daily_changes[i][1] == "icon-Caret-up" else "D") +  daily_changes[i][0]}
  </td>
  <td>
   {("U" if weekly_changes[i][1] == "icon-Caret-up" else "D") + weekly_changes[i][0]}
  </td>
 </tr>
  \n"""
print(tbody)


 <tr>
  <td>
   1
  </td>
  <td>
   Bitcoin | BTC
  </td>
  <td>
   $22,818.29
  </td>
  <td>
   D4.31%
  </td>
  <td>
   D0.05%
  </td>
 </tr>
  

 <tr>
  <td>
   2
  </td>
  <td>
   Ethereum | ETH
  </td>
  <td>
   $1,665.09
  </td>
  <td>
   D6.49%
  </td>
  <td>
   U3.58%
  </td>
 </tr>
  

 <tr>
  <td>
   3
  </td>
  <td>
   Tether | USDT
  </td>
  <td>
   $1.00
  </td>
  <td>
   U0.01%
  </td>
  <td>
   U0.00%
  </td>
 </tr>
  

 <tr>
  <td>
   4
  </td>
  <td>
   USD Coin | USDC
  </td>
  <td>
   $0.9997
  </td>
  <td>
   D0.03%
  </td>
  <td>
   U0.01%
  </td>
 </tr>
  

 <tr>
  <td>
   5
  </td>
  <td>
   BNB | BNB
  </td>
  <td>
   $319.76
  </td>
  <td>
   D1.89%
  </td>
  <td>
   U14.07%
  </td>
 </tr>
  

 <tr>
  <td>
   6
  </td>
  <td>
   Binance USD | BUSD
  </td>
  <td>
   $1.00
  </td>
  <td>
   D0.02%
  </td>
  <td>
   U0.01%
  </td>
 </tr>
  

 <tr>
  <td>
   7
  </td>
  <td>
   XRP | XRP
  </td>
  <td>
   $0.363
  </td>
  <td>
   D4.38%
  </td>
  <td>
   D1.48%
  

In [66]:
content = f"""
 <table>
  <tr>
   <th>Ranking</th>
   <th>Currency</th>
   <th>Price</th>
   <th>24h change</th>
   <th>7d change</th>
  </tr>
  {tbody}
 </table>
"""

In [67]:
#SMTPLIB email sender
port = 587
smtp_server = "smtp.gmail.com"
sender_email = "DON'T"
receiver_email = "LOOK HERE"
password = "PLEASE"

message = MIMEMultipart("message")
message["Subject"] = "UpTime Update"
message["From"] = sender_email
message["To"] = receiver_email

html_message = f"""\
 <html>
  <body>
   {content}
  </body>
 </html>
 """

message_object = MIMEText(html_message, "html")

message.attach(message_object)

context = ssl.create_default_context()
with smtplib.SMTP(smtp_server, port) as server:
 server.ehlo()
 server.starttls(context = context)
 server.ehlo()
 server.login(sender_email, password)
 server.sendmail(sender_email, receiver_email, message.as_string())